In [1]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [2]:
sys.path.append('../..')

In [3]:
from lung_cancer.dataset import FilesIndex, Dataset
from lung_cancer import CTImagesMaskedBatch as CTIMB
from lung_cancer import update_histo_by_lunaset, split_dump_lunaset
from lung_cancer import sample_ellipsoid_region

### Prepare histo for random cropping

* csv with nodules

In [4]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')
nodules.head(2)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708


* Set params of ellispod peel, generate sample from the peel

In [5]:
CENTER_PEEL = (200, 270, 250)

# by axes we understand semi-axes
AXES_PEEL = (100, 110, 150)
MULT_RANGE = (0.7, 1.3)
SIZE = 10000

# shape of scans afte runify spacing
from lung_cancer.pipelines.ppl import USPACING_SHAPE

* set prob of elliptic component in mixture

In [6]:
PROB_PEEL = 0.2
weight = PROB_PEEL / (1 - PROB_PEEL) * len(nodules) / SIZE

In [7]:
sample = sample_ellipsoid_region(axes=AXES_PEEL, center=CENTER_PEEL, mult_range=MULT_RANGE, size=SIZE)

* Create a histo using sample coords from elliptic peel

In [8]:
ranges = list(zip([0]*3, USPACING_SHAPE))
histo = list(np.histogramdd(sample, range=ranges, bins=6))

# weight the histo so that mixture be with chosen probs
histo[0] *= weight

* Update histo using pipeline

In [9]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
lunaix = FilesIndex(path=DIR_LUNA, no_ext=True)
lunaset = Dataset(index=lunaix, batch_class=CTIMB)

In [12]:
lunaset.cv_split(0.9, shuffle=True)

In [14]:
np.sum(histo[0])

296.5

In [15]:
ppl = update_histo_by_lunaset(lunaset, nodules, histo)
# ppl.run()

In [20]:
np.sum(histo[0])

1482.5

In [17]:
len(ppl)

888

* Save the histo

In [18]:
import pickle

#with open('histo.pkl', 'wb') as file:
 #   pickle.dump(histo, file)

* Read the histo

In [19]:
with open('histo.pkl', 'rb') as file:
    histo = pickle.load(file)

### Run pipeline that dumps two cropsets

In [22]:
len(lunaset.train)

799

* Dump indices of the train subset

In [29]:
#with open('lunaix_train.pkl', 'wb') as file:
 #   pickle.dump(lunaset.train.indices, file)

In [30]:
DIR_DUMP = '/data/lunaset_split/train/'
ppl = split_dump_lunaset(dir_cancer=DIR_DUMP + 'cancer/', dir_ncancer=DIR_DUMP + 'ncancer/',
                         lunaset=lunaset.train, nodules_df=nodules, histo=histo)

In [ ]:
ppl.run()

selector_events.py[LINE:53]#DEBUG    [2017-09-07 15:30:16,287]  Using selector: EpollSelector


In [43]:
len(os.listdir(DIR_DUMP + 'ncancer/'))

102999